# Exercise 5


_course: quantum cryptography for beginners
<br>date: 21 september 2020
<br>author: burton rosenberg_

###  Kronecker Product

In a classical circuit, a large number can be given by a short description. A collection of bits thought of as working together on a number or symbol representation is called a _register_. An $n$ bit classical register can be in on of $2^n$ possible states, all possible 0/1 combinations. For instance, $n=3$:

$$
000, 001, 010, 011, 100, 101, 110, 111
$$

And these combinations more than naturally correspond to numbers, but placing each bit in the appropriate position on a base-2 number representation scheme. For instance, a bit pattern to represent the number $5$,

$$
101 \rightarrow  1, 0, 1 \rightarrow 1\cdot 2^2 + 0\cdot 2^1 + 1 \cdot 2^0 = 4 + 1 = 5
$$

In a qubit, their is a degree of freedom attached to each combination of bits, not only to each bit. And it one must understant that a $n$ bit quantum register as $2^n$ possible state entries, and each entry is a complex number, not simply 0/1.

Therefore, when qubits are combined into a register they create a tensor product of the original state that fills and exponentially large space. And this is done in linear algebra using an operation called the Kronkecker product.

The kronecker product is a very simple operation, $a \otimes b$, replace by each $a_i$ and entire copy of $b$ multiplied by $a_i$,

$$
[x,\;y] \otimes [s,\;t] = [x \cdot[s,\;t],\; y\cdot[s,\;t] ] = [xs,\;xt,\;ys,\;yt]
$$

This has the very simple effect of expressing $n$ as the tensor product of several $|0\rangle = [1,0]$ or $|1\rangle = [0,1]$ qubits that end up placing a single 1 in the n-th location of an otherwise all zero vector.



In [1]:
import math
import random
import numpy as np

np.set_printoptions(precision=2,floatmode='fixed',suppress=True)

def qubit_zero():
    """
    returns a |0>, as a linear algebra object
    """
    return np.array([1.0,0.0])

def qubit_one():
    """
    returns a |1>, as a linear algebra object
    """
    return np.array([0.0,1.0])

#
# helper functions
#

def int_to_binary_list(n,n_bits):
    """
    given an positive int n, and and a number of bits n_bits, sufficient
    to convert int n, return an n element list of 0 and 1, the binary 
    representation of n. The first element in the list is the least 
    significant bit.
    """
    z = np.zeros(n_bits)
    for i in range(n_bits):
        if n%2!=0:
            z[i] = 1
        n = n//2
    return z
 

def qubit_n(n,n_bits):
    """
    return the state vector |n>
    """
    
    def bit_to_qubit(b):
        if b==1:
            return qubit_one()
        return qubit_zero()
    
    z = int_to_binary_list(n,n_bits)
    
    phi = bit_to_qubit(z[0])
    for i in range(1,n_bits):
        phi = np.kron(bit_to_qubit(z[i]),phi)
        
    return phi
    
for n in range(8):
    print(f'|{n}> =',qubit_n(n,3))

|0> = [1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
|1> = [0.00 1.00 0.00 0.00 0.00 0.00 0.00 0.00]
|2> = [0.00 0.00 1.00 0.00 0.00 0.00 0.00 0.00]
|3> = [0.00 0.00 0.00 1.00 0.00 0.00 0.00 0.00]
|4> = [0.00 0.00 0.00 0.00 1.00 0.00 0.00 0.00]
|5> = [0.00 0.00 0.00 0.00 0.00 1.00 0.00 0.00]
|6> = [0.00 0.00 0.00 0.00 0.00 0.00 1.00 0.00]
|7> = [0.00 0.00 0.00 0.00 0.00 0.00 0.00 1.00]




The Kronecker Product applies to operators as well as states. For instance, 


\begin{eqnarray}
X \otimes Z &=& \left[  \begin{array}{cc} 0 & 1 \\ 1 & 0 \end{array} \right]
\otimes \left[  \begin{array}{cc} 1 & 0 \\ 0 & -1 \end{array} \right] \\
&=& \left[  \begin{array}{cc} 0\cdot
   \left[  \begin{array}{cc} 1 & 0 \\ 0 & -1 \end{array} \right]
   & 1 \cdot  \left[  \begin{array}{cc} 1 & 0 \\ 0 & -1 \end{array} \right]
   \\ 1 \cdot  \left[  \begin{array}{cc} 1 & 0 \\ 0 & -1 \end{array} \right]
   & 0 \cdot  \left[  \begin{array}{cc} 1 & 0 \\ 0 & -1 \end{array} \right]\end{array} \right] \\
   &=& \left[\begin{array}{cccc}
   0&0&1&0\\0&0&0&-1\\1&0&0&0\\0&-1&0&0
   \end{array}\right]
\end{eqnarray}


In applying tensor products of operators to tensor products of states, you get the same result if you first tensor product the operators and states with themselves, getting a single large matrix and a single long vector, and then applying operator-state; or individually applying each operator in the tensor product to the corresponding vector and once done, tensor producting the result,

$$
A \otimes B \; |\phi \, \psi\rangle = A\,|\phi\rangle \otimes B\,|\psi\rangle
$$

We give an example for $A=B=H$ on $|0\rangle$.


In [2]:
def operator_H_nn():
    """
    return the Hadamard operator
    """
    return np.array([[1.0,1.0],[1.0,-1.0]])

def apply_operator_to_state(operator,phi):
    """
    return operator applied to phi
    """
    return np.matmul(operator,phi)

def kronecker_product(a,b):
    """
    a and b can be either operators, such as H, or states, such as |0>.
    the numpy library has kron which applies to both cases
    """
    return np.kron(a,b)


def HH_00_tensor_last():
    """
    return (H(x)H) (|00>)
    """
    phi = apply_operator_to_state(operator_H_nn(),qubit_zero())
    return kronecker_product(phi,phi)


def HH_00_operate_last():
    """
    return (H |0>) (x) (H |0>)
    """
    hh = kronecker_product(operator_H_nn(),operator_H_nn())
    oo = kronecker_product(qubit_zero(),qubit_zero())
    return apply_operator_to_state(hh,oo)

def test_exercise_z_1():
    phi_1 = HH_00_tensor_last()
    phi_2 = HH_00_operate_last()
    print("H|0> (x) H|0> =\t", phi_1)
    print("H(x)H |00> =\t", phi_2)
    if np.allclose(phi_1,phi_2):
        print('they are the same')
        return True
    else:
        print('they are different')
    return False

print(test_exercise_z_1())

H|0> (x) H|0> =	 [1.00 1.00 1.00 1.00]
H(x)H |00> =	 [1.00 1.00 1.00 1.00]
they are the same
True


### Exercise A:


Compute the state of $H^{\otimes n}\,|i\rangle$ for the case $n$ and $i = 0, 1, \ldots , 7$. 

_Note:_ This is an exercise in the Kroneker product/tensor product. We do not build any circuit just yet.



In [6]:

def HHH_i_experimental(i): 
    """
    given a 3 bit integer i, return H(x)H(x)H |i>
    you may calculate this either tensor last or apply operator last.
    """
    
    # <---- remove this code and write your own
    phi = np.zeros(8)
    # <---- remove this code and write your own
    
    return phi

for i in range(8):    
    print(f'|{i}> -->', HHH_i_experimental(i))
    


|0> --> [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
|1> --> [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
|2> --> [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
|3> --> [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
|4> --> [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
|5> --> [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
|6> --> [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]
|7> --> [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]


### Exercise B

If we were to build a quantum circuit for exericse A, and measure the 3 qubits, what would be the resulting observation?

In [4]:
print(f'i    probability of H^(3) |i> == |j>')
print(f'     j=|0>  j=|1>  j=|2>  j=|3>  j=|4>  j=|5>  j=|6>  j=|7>')
print(f'-----------------------------------------------------------')
for i in range(8):
    print(f'{i}     ?      ?      ?      ?      ?      ?      ?      ?')

i    probability of H^(3) |i> == |j>
     j=|0>  j=|1>  j=|2>  j=|3>  j=|4>  j=|5>  j=|6>  j=|7>
-----------------------------------------------------------
0     ?      ?      ?      ?      ?      ?      ?      ?
1     ?      ?      ?      ?      ?      ?      ?      ?
2     ?      ?      ?      ?      ?      ?      ?      ?
3     ?      ?      ?      ?      ?      ?      ?      ?
4     ?      ?      ?      ?      ?      ?      ?      ?
5     ?      ?      ?      ?      ?      ?      ?      ?
6     ?      ?      ?      ?      ?      ?      ?      ?
7     ?      ?      ?      ?      ?      ?      ?      ?



### Exercise C:

Discover and code the function that determines the sign for $|j\rangle$ in the superposition $H^{\otimes n}\,|i\rangle$.

That is, we would like to calculate directly $f(i,j)$ a 0/1 valued function that gives the sign pattern you saw in exercise A,

$$
|i\rangle = \sum_j (-1)^{f(i,j)} \;|j\rangle
$$

Here is a example, 

\begin{eqnarray}
H^{\otimes 3}\,|5\rangle &=& H^{\otimes 3}\,|101\rangle \\
&=& H\,|1\rangle \otimes H\,|0\rangle \otimes H\,|1\rangle \\
&=& |-\rangle \otimes |+\rangle \otimes |-\rangle \\
&=& \bigl( |0\rangle - |1\rangle \bigr) \otimes \bigl( |0\rangle + |1\rangle \bigr) 
\otimes \bigl( |0\rangle - |1\rangle \bigr) \\
&=& |000\rangle - |001\rangle + |010\rangle - |011\rangle - |100\rangle +|101\rangle - |110\rangle + |111\rangle \\
&=& |0\rangle - |1\rangle + |2\rangle - |3\rangle - |4\rangle +|5\rangle - |6\rangle + |7\rangle \\
\end{eqnarray}

_Hint:_ Consider the binary representations of $i$ and $j$. Only bits that are 1 in $i$ can create the possibility for negative coefficients; and the coefficient will only be negative if the number of 1's in $j$ are odd.

So for $|5\rangle$, the coefficient is $-1$ only if one but not both of the end bits i the binary represention of $j$ are 1.



In [8]:

def mod_2_dot_prod(i,j,n_bits):
    # int_to_binary_list was defined above
    iz = int_to_binary_list(i,n_bits)
    jz = int_to_binary_list(j,n_bits)
    c = 0 ;
    for ie, je in zip(iz,jz):
        c += ie*je
    return c % 2 
   
    
def f(i,j):
    
    # <---- phoney code
    f = 0
    # -----> phoney code
    
    return f

def HHH_i_formula(i):
    """
    from i return HHH |i>, using the formula for f(i,j)
    """
    n_bits = 3
    v = np.zeros(2**n_bits)
    for j in range(2**n_bits):
        
        # <---- phoney code
        v = np.ones(2**n_bits)
        # ----> phoney code
        
    return v
        
        
def HHH_i_experimental_formula_test():
    for n in range(8):
        phi_1 = HHH_i_experimental(n)
        phi_2 = HHH_i_formula(n)
        print(f'|{n}> =', phi_2)
        if np.allclose(phi_1,phi_2):
            pass
        else:
            return False
    return True

if HHH_i_experimental_formula_test():
    print('passed test')
else:
    print('did not pass test')

|0> = [1.00 1.00 1.00 1.00 1.00 1.00 1.00 1.00]
did not pass test


## Bernstein-Vazirani

We are given a function-box $U_r(x)$ that has a hidden $n$ bit number $r$ and computes a 0/1 output on an $n$ bit input $x$. We are promised that the function is the mod 2 inner product of the bit vectors of the bits of $r$ and $x$,

\begin{eqnarray}
U_r(x) &=& \bigoplus_{i=0}^n r_i\cdot x_i,
\end{eqnarray}


where $x = x_{n-1}x_{n-2}\ldots x_0$ and $r = r_{n-1}r_{n-2}\ldots r_0$ are the bits in the binary representation of the numbers $r$ and $x$, respectively. 

The problem statement: Find $r$.


Classically, we must have $n$ queries of $U_r(x)$. There are $n$ bit of truely independent and distinct information in $r$, and we can extract only one bit of information with each query. We could try $x=1, 2, 4, \ldots$, to extract $r_i$ one by one, but there are other strategies. However no strategy in classical computation can take less then $n$ queries to discover fully the value $r$.

However, in quantum computation, a single $n$ bit measure can extract the answer, even though the computation of $U_r(x)$ gives only a single bit. 

We have a 0,1 function $f(i,j)$ such that,

$$
H^{\otimes n}\,|i\rangle = \sum_j (-1)^{f(i,j)} |j\rangle
$$

Applying $H^{\otimes n}$ to both sides, then

$$
H^{\otimes n}\, \sum_j (-1)^{f(i,j)} |j\rangle = |i\rangle
$$

The idea of Bernstein-Vazirani is to calculate $f(i,j)$ by the quantum circuit $U_i(j)$, simulatneously for all $j$, and transfer the $0/1$ output of $U_i(j)$ as the $+1/{-}1$ sign on $|j\rangle$.

#### Implementing U

To the $n$ bit register for the state $|i\rangle$, we add a ancilla qubit that is in the the $|-\rangle$ state. We attach CNOT's to control this ancilla by the bits in $i$ that are 1. Then if the bit in $j$ is also a one, the CNOT applies X to the ancilla, else it does nothing to the ancilla.

$$
U_{i_s}\,|j_s\rangle = \begin{cases} X  &\mbox{ if } i_s\cdot j_s= 1\\
I&\mbox{ any other case} 
\end{cases}
$$

so

$$
U_{i_s}\,|j_s\rangle\,|-\rangle = \begin{cases} -1\cdot |-\rangle  &\mbox{ if } i_s\cdot j_s= 1\\
+1\cdot |-\rangle &\mbox{ any other case} 
\end{cases}
$$

with the ultimate effect on the ancilla, for each $j$,

$$
U_i(j)\,|-\rangle = \otimes_s U_{i_s}\,|j_s\rangle\,|-\rangle =(-1)^{f(i,j)} |-\rangle.
$$

#### Phase kickback

Also note that because of how the Kronecker product works, although this flipping of sign seems to apply only to the ancilla, it factors out and applies to the entire state. This is called phase kickback, because an effect on the controlled bit is kicked back onto the controlling bit (on all the bits). If we then ignore the ancilla, we have remaining,

$$
U_i(j)\,|j\rangle = (-1)^{f(i,j)}\,|j\rangle
$$

#### Inverse Hadamard

If we then apply Hadamard to the superposed state over all $j$,


$$
H^{\otimes n}\sum_j U_i(j)\,|j\rangle = H^{\otimes n}\sum_j (-1)^{f(i,j)}\,|j\rangle = |j\rangle
$$

which was the $r$ in the original notation. This completes the algorithm description.

Note that, we do read $n$ bits as needed to know the $n$ bit value $r$. We should not be surprised, because we simultaneously evaluated $U_r(x)$ on all $2^n$ possible values of $x$, and that is a lot of information.

This does not yet solve the problem, since we now need to extract from this superposition the value $r$. We have at this point $H^{\otimes n}|j\rangle$ and if we were to measure it we would get an equally observation from $|0\rangle$ through $|2^n-1\rangle$, which tells us nothing of a particular $|j\rangle$.

We use the final bank of Hadamards to interfer the plus and minus signs that would otherwise be lost in the absolute value of probabilities, to distil out in a single quantum operation the particular pattern of signs that leads to the answer.


__Example:__

$$
H|1\rangle = |0\rangle - |1\rangle + |2\rangle - |3\rangle + |4\rangle -|5\rangle + |6\rangle - |7\rangle 
$$

The pattern is all states with an odd number are negative. So we can use the least significant qubit as control on the acilla to flip the sign for exactly thise qubits.


In [10]:
import qiskit
import time, math

from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.jobstatus import JOB_FINAL_STATES, JobStatus
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

qiskit.__qiskit_version__

args_g = []

# your api token from IBM, first time run.
# after that None is good

#api_token = 'abcdefghijklmnopqrstuvwxyz'
api_token = None 

def load_or_save_IBMQ_account(api_token=None):
    global args_g
    if api_token:
       
    # only needs to be done once
        # then is stored in e.g. ~/.qistkit/qiskitrc
        IBMQ.save_account(api_token)
    provider = IBMQ.load_account()
    return provider

def list_backends(provider):
    global args_g
    backends = provider.backends()
    print('backends available:')
    for be in backends:
        st = be.status()
        if st.operational:
            print(f'\t{be.name()}, pending jobs:{st.pending_jobs}')

            
def run_quantum_circuit_on_backend(quantum_circuit,provider,backend):
    backend = provider.get_backend(backend)
    qobj = assemble(transpile(quantum_circuit, backend=backend), backend=backend)
    job = backend.run(qobj)
    return job


def wait_for_job(backend, job, wait_interval=5):
    backend = provider.get_backend(backend)
    retrieved_job = backend.retrieve_job(job.job_id())
    start_time = time.time()
    job_status = job.status()
    while job_status not in JOB_FINAL_STATES:
        print(f'Status @ {time.time() - start_time:0.0f} s: {job_status.name},'
              f' est. queue position: {job.queue_position()}')
        time.sleep(wait_interval)
        job_status = job.status()


print("listing backends ...")
provider = load_or_save_IBMQ_account(api_token)
list_backends(provider)

# choose your backend

backend = 'ibmq_qasm_simulator'
#backend = 'ibmq_armonk'
#backend = 'ibmq_vigo'
#backend = 'ibmq_london'
#backend = 'ibmq_ourense'

# and so forth ... chose from the results given by provider.backends()

listing backends ...


/Users/ojo/anaconda3/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


backends available:
	ibmq_qasm_simulator, pending jobs:1
	ibmqx2, pending jobs:12
	ibmq_16_melbourne, pending jobs:1
	ibmq_vigo, pending jobs:3
	ibmq_ourense, pending jobs:6
	ibmq_valencia, pending jobs:1
	ibmq_armonk, pending jobs:24
	ibmq_santiago, pending jobs:29



### Exercise D: 

Create the described circuit for all values 0 through 7 in a qubit register. You should measure the value $|i\rangle$ with 100% probability in your $i$-th circuit. 



In [11]:

def b_v_prepare_initial_n(n):
    qreg_q = QuantumRegister(4, 'q')
    creg_c = ClassicalRegister(4, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)

    for i in range(3):
        if n%2==1:
            circuit.x(qreg_q[i])
        n = n//2
    
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.h(qreg_q[2])
    circuit.x(qreg_q[3])
    circuit.h(qreg_q[3])
    circuit.barrier()
    
    return circuit

def b_v_prepare_initial():
    return b_v_prepare_initial_n(0)
    
def b_v_uxor_0(circuit):
    return circuit

def b_v_uxor_1(circuit):
    qreg_q = circuit.qubits
    circuit.cx(qreg_q[0], qreg_q[3])
    return circuit

def b_v_uxor_2(circuit):
    
    # <---> write here
    
    return circuit

def b_v_uxor_4(circuit):
    
    # <---> write here
    
    return circuit

def b_v_uxor_3(circuit):
     
    # <---> write here
    
    return circuit

def b_v_uxor_5(circuit):
    
    # <---> write here
    
    return circuit

def b_v_uxor_6(circuit):
    
    # <---> write here
    
    return circuit

def b_v_uxor_7(circuit):
    
    # <---> write here
    
    return circuit

def b_v_transform_and_measure(circuit):
    qreg_q = circuit.qubits
    creg_c = circuit.clbits
    circuit.barrier()
    circuit.h(qreg_q[0])
    circuit.h(qreg_q[1])
    circuit.h(qreg_q[2])
    circuit.h(qreg_q[3])
    circuit.x(qreg_q[3])
    circuit.barrier()
    circuit.measure(qreg_q[0], creg_c[0])
    circuit.measure(qreg_q[1], creg_c[1])
    circuit.measure(qreg_q[2], creg_c[2])
    circuit.measure(qreg_q[3], creg_c[3])

    print('\n-------- CIRCUIT ---------')
    print(circuit.draw(output='text'))
    print('-------------------------\n')

    return circuit

def b_v_construct(uxor):
    return b_v_transform_and_measure(uxor(b_v_prepare_initial()))

uxors = [b_v_uxor_0,
         b_v_uxor_1,
         b_v_uxor_2,
         b_v_uxor_3,
         b_v_uxor_4,
         b_v_uxor_5,
         b_v_uxor_6,
         b_v_uxor_7]

def b_v_test(uxors):

    # fix this to actually get the answer from result
    
    for i in range(len(uxors)):
        circuit = b_v_construct(uxors[i])
        job = run_quantum_circuit_on_backend(circuit,provider,backend)
        print(f'results: waiting for results from backend {backend} ...')
        wait_for_job(backend, job)
        result = job.result()
        counts = result.get_counts()
        print(f'results: {result.get_counts()}')
        max_key = max(counts,key=counts.get)
        max_key_num = int(max_key,2)
        shots = sum(counts.values())
        if i==max_key_num:
            print(f'result correct: {max_key_num}, shots: {counts[max_key]} of {shots}')
        else:
            print('result not correct')

b_v_test(uxors)


-------- CIRCUIT ---------
     ┌───┐      ░  ░ ┌───┐      ░ ┌─┐         
q_0: ┤ H ├──────░──░─┤ H ├──────░─┤M├─────────
     ├───┤      ░  ░ ├───┤      ░ └╥┘┌─┐      
q_1: ┤ H ├──────░──░─┤ H ├──────░──╫─┤M├──────
     ├───┤      ░  ░ ├───┤      ░  ║ └╥┘┌─┐   
q_2: ┤ H ├──────░──░─┤ H ├──────░──╫──╫─┤M├───
     ├───┤┌───┐ ░  ░ ├───┤┌───┐ ░  ║  ║ └╥┘┌─┐
q_3: ┤ X ├┤ H ├─░──░─┤ H ├┤ X ├─░──╫──╫──╫─┤M├
     └───┘└───┘ ░  ░ └───┘└───┘ ░  ║  ║  ║ └╥┘
c: 4/══════════════════════════════╩══╩══╩══╩═
                                   0  1  2  3 
-------------------------

results: waiting for results from backend ibmq_qasm_simulator ...
Status @ 0 s: VALIDATING, est. queue position: None
results: {'0000': 1024}
result correct: 0, shots: 1024 of 1024

-------- CIRCUIT ---------
     ┌───┐      ░       ░ ┌───┐      ░ ┌─┐         
q_0: ┤ H ├──────░───■───░─┤ H ├──────░─┤M├─────────
     ├───┤      ░   │   ░ ├───┤      ░ └╥┘┌─┐      
q_1: ┤ H ├──────░───┼───░─┤ H ├──────░──╫─┤M├──────
     ├───┤ 


### Exercise E: 

In execise D we started with the superposition $H^{\otimes n} |0\rangle$ (and $|0\rangle$ was understood to be $n$-bits, something that is not obvious in the notation).

What would happen if we starte off with $H^{\otimes n} |j\rangle$ for $j$ any state between $0$ and $2^n-1$. (These are the $2^n$ non-superposed statess)?

Predict, make a model in your head to guide your prediction, create some experiments to test your guess.
